<a href="https://colab.research.google.com/github/mapmatters/review_insight/blob/main/1nd_trial_using_sample_set_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/master_degree/paper')
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 논문계획서_2nd.gdoc			  data
 archive.zip				 '기존 분석결과.gdoc'
 booking.com_hotel_review_korea.csv.zip  '논문 지도교수 배정신청서.gdoc'
 code_R					 '레퍼런스 링크.gdoc'
 colab					  model


In [3]:
import pandas as pd
import pickle
import json
from io import StringIO
import sys

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords') 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
# Language detection 등을 위한 라이브러리 import
!pip install fasttext

     |████████████████████████████████| 71kB 5.1MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3039069 sha256=094c4092b76b750c81eea549de486a22f1797438f79d4b19d9dc075230b33dba
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [26]:
import fasttext
!wget -O /tmp/lid.176.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
path_to_pretrained_model = '/tmp/lid.176.bin'

text2 = 'In 1793, Alexander Hamilton recruited Webster to move to New York City and become an editor for a Federalist Party newspaper.'
fmodel = fasttext.load_model(path_to_pretrained_model)
fmodel.predict([text2])  # ([['__label__en']], [array([0.9331119], dtype=float32)]

--2021-01-17 00:07:54--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘/tmp/lid.176.bin’

/tmp/lid.176.bin    100%[===================>] 125.18M  62.7MB/s    in 2.0s    

2021-01-17 00:07:56 (62.7 MB/s) - ‘/tmp/lid.176.bin’ saved [131266198/131266198]



([['__label__en']], [array([0.9331119], dtype=float32)])

In [28]:
# stopwords 점검
original_stops = stopwords.words('english')
print(len(original_stops))
my_stops = ['zoom', 'fbm'] + original_stops
print(my_stops)

179
['zoom', 'fbm', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'sam

In [29]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    # keywords= [lemma for lemma in lemmas if lemma not in my_stops] # apply customized stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords

def build_freqs(text):
    freqs = {}
    for word in preprocess_text(text):
        if word in freqs:
            freqs[word] += 1
        else:
            freqs[word] = 1
    return freqs

# >>> l = ["a","b","b"]
# >>> [[x,l.count(x)] for x in set(l)]
# [['a', 1], ['b', 2]]
# >>> dict((x,l.count(x)) for x in set(l))
# {'a': 1, 'b': 2}


# Dataset 점검

In [30]:
df = pd.read_pickle('data/processed/rv_en_2.pkl')
df.head(10)

,app_nm,review,rating,dt,app_ver,lang,keywords,detoken,items
0,zoom,ভাল,2.0,2020-12-28,5.2.45092.0831,en,"[ভ, ল]",ভ ল,"{'ভ': 1, 'ল': 1}"
1,zoom,This is our one of the education thing. We can...,5.0,2020-12-26,Unknown,en,"[one, education, thing, cant, unlike, never]",one education thing cant unlike never,"{'one': 1, 'education': 1, 'thing': 1, 'cant':..."
2,zoom,Good,5.0,2020-12-26,5.4.6.812,en,[good],good,{'good': 1}
3,zoom,Worst appp,2.0,2020-12-28,Unknown,en,"[worst, appp]",worst appp,"{'worst': 1, 'appp': 1}"
4,zoom,Good,4.0,2020-12-28,5.4.7.946,en,[good],good,{'good': 1}
5,zoom,Pro,5.0,2020-12-26,5.2.42588.0803,en,[pro],pro,{'pro': 1}
6,zoom,It's very very useful for school students,5.0,2020-12-26,5.4.7.946,en,"[useful, school, students]",useful school students,"{'useful': 1, 'school': 1, 'students': 1}"
7,zoom,Max 4 people on one screen including myself. T...,1.0,2020-12-23,Unknown,en,"[max, 4, people, one, screen, include, usable]",max 4 people one screen include usable,"{'max': 1, '4': 1, 'people': 1, 'one': 1, 'scr..."
8,zoom,Super,5.0,2020-12-23,5.4.4.615,en,[super],super,{'super': 1}
9,zoom,Won't let me Unmute. . The level of Simplicity...,3.0,2020-12-24,Unknown,en,"[win, let, unmute, level, simplicity, complicate]",win let unmute level simplicity complicate,"{'win': 1, 'let': 1, 'unmute': 1, 'level': 1, ..."


In [31]:
print(df['app_nm'].unique())
print('Number of apps: ',len(df['app_nm'].unique()))

['zoom' 'discord' 'fbm']
Number of apps:  3


In [32]:
print('rows and columns', df.shape, '\n')
print('reviews of each app', df.groupby(['app_nm']).size())
print('reviews of each app', df.groupby(['app_nm']).count())

rows and columns (1250624, 9) 

reviews of each app app_nm
discord    118292
fbm        795610
zoom       336722
dtype: int64
reviews of each app          review  rating      dt  app_ver    lang  keywords  detoken   items
app_nm                                                                     
discord  118292  118292  118292   118292  118292    118292   118292  118292
fbm      795610  795610  795610   795610  795610    795610   795610  795610
zoom     336722  336722  336722   336722  336722    336722   336722  336722


In [43]:
# filter out non-english
df['lang'] = df['detoken'].apply(lambda x: fmodel.predict(x))
df['lang2'] = df['lang'].apply(lambda x: '__label__en' in list(x)[0])
df = df[df['lang2']]
df

,app_nm,review,rating,dt,app_ver,lang,keywords,detoken,items,lang2
1,zoom,This is our one of the education thing. We can...,5.0,2020-12-26,Unknown,"((__label__en,), [0.8765208721160889])","[one, education, thing, cant, unlike, never]",one education thing cant unlike never,"{'one': 1, 'education': 1, 'thing': 1, 'cant':...",True
2,zoom,Good,5.0,2020-12-26,5.4.6.812,"((__label__en,), [0.8952815532684326])",[good],good,{'good': 1},True
3,zoom,Worst appp,2.0,2020-12-28,Unknown,"((__label__en,), [0.6274816393852234])","[worst, appp]",worst appp,"{'worst': 1, 'appp': 1}",True
4,zoom,Good,4.0,2020-12-28,5.4.7.946,"((__label__en,), [0.8952815532684326])",[good],good,{'good': 1},True
6,zoom,It's very very useful for school students,5.0,2020-12-26,5.4.7.946,"((__label__en,), [0.9363648891448975])","[useful, school, students]",useful school students,"{'useful': 1, 'school': 1, 'students': 1}",True
...,...,...,...,...,...,...,...,...,...,...
1250618,fbm,You can't see pictures and videos you have sen...,3.0,2020-12-24,294.0.0.24.129,"((__label__en,), [0.8736512064933777])","[see, picture, videos, send, anymore, set, nic...",see picture videos send anymore set nickname,"{'see': 1, 'picture': 1, 'videos': 1, 'send': ...",True
1250619,fbm,problem in archive story and how many other se...,1.0,2020-12-25,290.0.0.16.119,"((__label__en,), [0.9673799872398376])","[problem, archive, story, many, see]",problem archive story many see,"{'problem': 1, 'archive': 1, 'story': 2, 'many...",True
1250620,fbm,why isn't there a way to change an individual'...,1.0,2020-12-28,294.0.0.24.129,"((__label__en,), [0.695924699306488])","[way, change, individual, notification, sound,...",way change individual notification sound whole...,"{'way': 1, 'change': 2, 'individual': 1, 'noti...",True
1250621,fbm,The latest update ruined this app... The chat ...,1.0,2020-12-28,294.0.0.24.129,"((__label__en,), [0.7956023812294006])","[latest, update, ruin, app, chat, bubble, suck...",latest update ruin app chat bubble suck notifi...,"{'latest': 1, 'update': 1, 'ruin': 1, 'app': 1...",True


In [44]:
# len(df['keywords'][0])
df['n_words'] = df['keywords'].apply(lambda x: len(x))
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,app_nm,review,rating,dt,app_ver,lang,keywords,detoken,items,lang2,n_words
1,zoom,This is our one of the education thing. We can...,5.0,2020-12-26,Unknown,"((__label__en,), [0.8765208721160889])","[one, education, thing, cant, unlike, never]",one education thing cant unlike never,"{'one': 1, 'education': 1, 'thing': 1, 'cant':...",True,6
2,zoom,Good,5.0,2020-12-26,5.4.6.812,"((__label__en,), [0.8952815532684326])",[good],good,{'good': 1},True,1
3,zoom,Worst appp,2.0,2020-12-28,Unknown,"((__label__en,), [0.6274816393852234])","[worst, appp]",worst appp,"{'worst': 1, 'appp': 1}",True,2
4,zoom,Good,4.0,2020-12-28,5.4.7.946,"((__label__en,), [0.8952815532684326])",[good],good,{'good': 1},True,1
6,zoom,It's very very useful for school students,5.0,2020-12-26,5.4.7.946,"((__label__en,), [0.9363648891448975])","[useful, school, students]",useful school students,"{'useful': 1, 'school': 1, 'students': 1}",True,3
...,...,...,...,...,...,...,...,...,...,...,...
1250618,fbm,You can't see pictures and videos you have sen...,3.0,2020-12-24,294.0.0.24.129,"((__label__en,), [0.8736512064933777])","[see, picture, videos, send, anymore, set, nic...",see picture videos send anymore set nickname,"{'see': 1, 'picture': 1, 'videos': 1, 'send': ...",True,7
1250619,fbm,problem in archive story and how many other se...,1.0,2020-12-25,290.0.0.16.119,"((__label__en,), [0.9673799872398376])","[problem, archive, story, many, see]",problem archive story many see,"{'problem': 1, 'archive': 1, 'story': 2, 'many...",True,5
1250620,fbm,why isn't there a way to change an individual'...,1.0,2020-12-28,294.0.0.24.129,"((__label__en,), [0.695924699306488])","[way, change, individual, notification, sound,...",way change individual notification sound whole...,"{'way': 1, 'change': 2, 'individual': 1, 'noti...",True,24
1250621,fbm,The latest update ruined this app... The chat ...,1.0,2020-12-28,294.0.0.24.129,"((__label__en,), [0.7956023812294006])","[latest, update, ruin, app, chat, bubble, suck...",latest update ruin app chat bubble suck notifi...,"{'latest': 1, 'update': 1, 'ruin': 1, 'app': 1...",True,14


In [48]:
# 3글자 이하 keyword 제거하기
df['keywords_2'] = df['keywords'].apply(lambda x: [word for x in x if len(word)>3])

NameError: ignored

In [14]:
# df[['n_words']].groupby(['n_words']).size()
df_model = df[df['n_words']>20]
print("\ngroup by n_words", df_model.groupby(['n_words']).size())
print("\ngroup by rating", df_model.groupby(['rating']).size())
print(df_model.shape)

df_model_1 = df_model[df_model['rating']==1.0]
df_model_5 = df_model[df_model['rating']==5.0]
print("\nrating 1: ", df_model_1.shape)
print("\nrating 5: ", df_model_5.shape)


group by n_words n_words
21     3680
22     3384
23     2860
24     2445
25     2151
       ... 
96        1
105       1
108       1
109       1
127       1
Length: 72, dtype: int64

group by rating rating
1.0    13512
2.0     4131
3.0     4225
4.0     4070
5.0     6860
dtype: int64
(32798, 10)

rating 1:  (13512, 10)

rating 5:  (6860, 10)


#### Gensim 으로 동일한 과정 해보기

In [22]:
import gensim
from gensim import corpora
from gensim.models import LdaModel

In [17]:
tokenized_doc = df_model_5['keywords']
tokenized_doc

81         [सर, म, र, ब, इल, जगह, त, ह, फ, भ, ज, ल, ग, क,...
164        [kinda, fun, wish, u, could, make, ppls, scree...
222        [arooj, sajjad, r, 4r, d7, 4r7, f, f7, r7, gam...
499        [hi, friends, please, make, zoom, version, and...
866        [hi, share, screen, option, please, fix, mean,...
                                 ...                        
1249903    [hogy, beszélsz, másokal, ha, karantén, lenne,...
1249961    [က, ဇ, ပ, လက, ဆ, င, အ, ဖစ, ခမ, ကအရမ, သ, ဘ, ထ, ...
1250243    [five, star, para, saken, pero, sana, bumalik,...
1250417    [since, days, see, friends, list, face, diffic...
1250563    [gjbhhkb, ut, ng, kv, f, uf, hm, gy, či, h, hi...
Name: keywords, Length: 6860, dtype: object

In [27]:
dictionary = corpora.Dictionary(tokenized_doc)
print(dictionary[36])
print(len(dictionary))

ppls
27212


In [28]:
from collections import Counter

min_count = 20
word_counter = Counter((word for words in tokenized_doc for word in words))
removal_word_idxs = {
    dictionary.token2id[word] for word, count in word_counter.items()
    if count < min_count
}

dictionary.filter_tokens(removal_word_idxs)
dictionary.compactify()
print('dictionary size : %d' % len(dictionary)) # dictionary size : 10354

dictionary size : 1286


In [29]:
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 리뷰 출력. 첫번째 문서의 인덱스는 0

[(18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)]


In [30]:
NUM_TOPICS = 10 #20개의 토픽, k=20
lda_model = LdaModel(corpus, id2word=dictionary, num_topics=10)

In [31]:
topics = lda_model.print_topics(num_words=10)

In [32]:
for topic in topics:
    print(topic)
print(lda_model.print_topics())

(0, '0.025*"app" + 0.015*"call" + 0.014*"use" + 0.013*"messenger" + 0.013*"star" + 0.012*"chat" + 0.012*"love" + 0.012*"friends" + 0.011*"5" + 0.011*"ako"')
(1, '0.019*"app" + 0.015*"chat" + 0.014*"love" + 0.013*"use" + 0.011*"friends" + 0.011*"ng" + 0.011*"make" + 0.010*"messenger" + 0.010*"one" + 0.009*"new"')
(2, '0.018*"get" + 0.016*"time" + 0.016*"app" + 0.014*"good" + 0.014*"go" + 0.013*"one" + 0.013*"work" + 0.013*"please" + 0.011*"know" + 0.010*"say"')
(3, '0.019*"র" + 0.018*"ন" + 0.017*"ল" + 0.016*"ক" + 0.016*"প" + 0.016*"ব" + 0.015*"স" + 0.015*"য" + 0.014*"ত" + 0.014*"জ"')
(4, '0.023*"app" + 0.019*"use" + 0.015*"work" + 0.013*"love" + 0.012*"great" + 0.011*"would" + 0.010*"time" + 0.010*"thank" + 0.010*"like" + 0.010*"update"')
(5, '0.025*"app" + 0.023*"use" + 0.020*"messenger" + 0.018*"message" + 0.017*"chat" + 0.016*"friends" + 0.014*"great" + 0.013*"video" + 0.012*"call" + 0.009*"game"')
(6, '0.018*"na" + 0.014*"ko" + 0.012*"sa" + 0.012*"app" + 0.011*"love" + 0.009*"get" +

In [33]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [34]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(10,)
1.0
(10, 1286)
1.0


### LDA 시각화

In [47]:
!pip install pyLDAvis
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [43]:
prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

In [49]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot') #or plt.style.use('ggplot')
pyLDAvis.display(prepared_data)